In [1]:
# scvelo virtual environment

import scvelo as scv
import numpy as np
import scanpy as sc
import math

In [2]:
import subprocess

# List all installed packages with their versions
def list_installed_packages():
    result = subprocess.run(['pip', 'list'], stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

# List all installed packages with their versions in requirements format
def freeze_installed_packages():
    result = subprocess.run(['pip', 'freeze'], stdout=subprocess.PIPE)
    print(result.stdout.decode('utf-8'))

# Execute the functions
list_installed_packages()
freeze_installed_packages()


Package             Version
------------------- -------
anndata             0.10.7
appnope             0.1.4
array_api_compat    1.7.1
asttokens           2.4.1
click               8.1.7
comm                0.2.2
contourpy           1.2.1
cycler              0.12.1
debugpy             1.6.7
decorator           5.1.1
exceptiongroup      1.2.0
executing           2.0.1
fonttools           4.53.0
get-annotations     0.1.2
h5py                3.11.0
importlib_metadata  7.1.0
importlib_resources 6.4.0
ipykernel           6.29.3
ipython             8.18.1
jedi                0.19.1
joblib              1.4.2
jupyter_client      8.6.2
jupyter_core        5.7.2
kiwisolver          1.4.5
legacy-api-wrap     1.4
llvmlite            0.42.0
loompy              3.0.7
matplotlib          3.9.0
matplotlib-inline   0.1.7
natsort             8.4.0
nest_asyncio        1.6.0
networkx            3.2.1
numba               0.59.1
numpy               1.26.4
numpy-groupies      0.11.1
packaging           24.0


In [3]:
def generate_negative_binomial_matrix(mean_matrix, overdispersion):
    # Ensure overdispersion is positive
    if overdispersion <= 0:
        raise ValueError("Overdispersion parameter must be positive.")
    
    # Calculate the parameters r and p for each element in the matrix
    r_matrix = np.full(mean_matrix.shape, overdispersion)
    p_matrix = r_matrix / (r_matrix + mean_matrix)
    
    # Generate the negative binomial random variables for each element in the matrix
    negative_binomial_matrix = np.random.negative_binomial(r_matrix, p_matrix)
    
    return negative_binomial_matrix

In [4]:
n_obs = 1000
n_vars = 20
t_max = 25
alpha = 4
beta = .2
gamma = .1
noise_level = 3
seed = 520
max_thre = 100
overdispersion = 1 # the larger means the more like a Poisson

# generate S with Gaussian noise
data = scv.datasets.simulation(n_obs=n_obs,
                               n_vars=n_vars,
                               t_max=t_max,
                               alpha=alpha,
                               beta=beta,
                               gamma=gamma,
                               noise_level=noise_level,
                               random_seed=seed)
S = data.layers['spliced']
U = data.layers['unspliced']
data.layers['true_velocity'] = beta*U-gamma*S
S_lam = math.e**(math.log(max_thre)*S/np.max(S)) - 1
U_lam = math.e**(math.log(max_thre)*U/np.max(U)) - 1

np.random.seed(929)
S_poi = generate_negative_binomial_matrix(mean_matrix=S_lam,
                                          overdispersion=overdispersion)
U_poi = generate_negative_binomial_matrix(mean_matrix=U_lam,
                                          overdispersion=overdispersion)
data.layers['spliced'] = S_poi.copy()
data.layers['unspliced'] = U_poi.copy()
data.X = S_poi.copy() # need to make sure X is the same as spliced matrix

ZeroDivisionError: float division by zero

In [ ]:
# https://github.com/theislab/scvelo/issues/1212

scv.pp.log1p(data)
sc.pp.pca(data)
sc.pp.neighbors(data, n_pcs=30, n_neighbors=30)
scv.pp.moments(data, n_pcs=None, n_neighbors=None)
scv.tl.recover_dynamics(data,t_max=25)
scv.tl.velocity_graph(data,vkey="true_velocity")

In [ ]:
scv.pl.velocity_embedding_grid(data,
                               basis='pca',
                               color="true_t",
                               vkey="true_velocity",
                               arrow_length=2,
                               arrow_size=2,
                               min_mass=10)

In [ ]:
scv.pl.velocity_embedding_stream(data,
                               basis='pca',
                               color="true_t",
                               vkey="true_velocity")

Now for scVelo

In [ ]:
data2 = scv.datasets.simulation(n_obs=n_obs,
                               n_vars=n_vars,
                               t_max=t_max,
                               alpha=alpha,
                               beta=beta,
                               gamma=gamma,
                               noise_level=noise_level,
                               random_seed=seed)
S = data2.layers['spliced']
U = data2.layers['unspliced']
S_lam = math.e**(math.log(max_thre)*S/np.max(S)) - 1
U_lam = math.e**(math.log(max_thre)*U/np.max(U)) - 1

np.random.seed(929)
S_poi = generate_negative_binomial_matrix(mean_matrix=S_lam,
                                          overdispersion=overdispersion)
U_poi = generate_negative_binomial_matrix(mean_matrix=U_lam,
                                          overdispersion=overdispersion)
data2.layers['spliced'] = S_poi.copy()
data2.layers['unspliced'] = U_poi.copy()
data2.X = S_poi.copy() # need to make sure X is the same as spliced matrix

In [ ]:
data2.X[0:10,0:10]

In [ ]:
# List of quantiles you want to find
quantiles = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

# Find the quantiles of the flattened array
quantile_results = np.quantile(data2.X, quantiles)

print(f"The quantiles {quantiles} of the matrix are: {quantile_results}")

In [ ]:
# https://github.com/theislab/scvelo/issues/1212

scv.pp.log1p(data2)
sc.pp.pca(data2)
sc.pp.neighbors(data2, n_pcs=30, n_neighbors=30)
scv.pp.moments(data2, n_pcs=None, n_neighbors=None)

In [ ]:
scv.tl.recover_dynamics(data2)

In [ ]:
scv.tl.velocity(data2, mode='dynamical')

In [ ]:
scv.tl.velocity_graph(data2)

In [ ]:
scv.pl.velocity_embedding_grid(data2,
                               basis='pca',
                               color="true_t",
                               arrow_length=2,
                               arrow_size=2,
                               min_mass=10)

In [ ]:
scv.pl.velocity_embedding_stream(data2,
                               basis='pca',
                               color="true_t")

In [ ]:
scv.tl.velocity_confidence(data2)
keys = 'velocity_confidence'
scv.pl.scatter(data2, c=keys, cmap='coolwarm', perc=[5, 95])